In [ ]:
import os
import langchain
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools
import requests
import openai

# API Keys
os.environ["SERPAPI_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

# Models
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo')
gpt3 = OpenAI(model_name='text-davinci-003')

# Template Queries
FIRST_TASK = " Your first task is to figure out what your first task is, then come up with a plan on how to carry it out"
ALL_TASKS = " Your first task is to figure out what your tasks are, then plan how to do them"
FIRST_TASK = ""

In [ ]:
# Wolfram Alpha API
WOLF_ALPHA_APPID = ''
BASE_URL = 'http://api.wolframalpha.com/v1/spoken?appid=' + WOLF_ALPHA_APPID

def wolfAlphaQuery(query):
    split_query = query.split(' ')
    formatted_query = ''
    for i in range(len(split_query)):
        if i == len(split_query)-1:
            formatted_query += split_query[i] + '%3f'
            continue
        formatted_query += split_query[i] + '+'
    url = BASE_URL + '&i=' + formatted_query
    response = requests.get(url)
    if int(response.status_code) != 200:
        return f"Could not retrieve answer! (Status Code {response.status_code})"
    result = response.content.decode()
    
    return result

In [ ]:
# Function to list all tasks
def createTaskList(prompt):
    prompt = f"You are an task creation AI that uses the result of an execution agent to create new tasks with the following objective: {prompt}. Return the tasks as an array."
    response = openai.Completion.create(engine="text-davinci-003",prompt=prompt,temperature=0.5,max_tokens=100,top_p=1,frequency_penalty=0,presence_penalty=0)
    result = response.choices[0].text.strip().split('\n')
    textResult = ""
    for i in result:
        textResult += i + '\n'
    return textResult

In [ ]:
# Function to prioritize tasks
def prioritizeTaskList(tasks):
    prompt = f"""You are an task prioritization AI tasked with reprioritizing the following tasks: {tasks}. Do not add, remove or modify any tasks. Return the result as a numbered list, like:
    #. First task
    #. Second task"""
    response = openai.Completion.create(engine="text-davinci-003",prompt=prompt,temperature=0.5,max_tokens=100,top_p=1,frequency_penalty=0,presence_penalty=0)
    result = response.choices[0].text.strip().split('\n')
    textResult = ""
    for i in result:
        textResult += i + '\n'
    return textResult


In [ ]:
# Execute each task separately
def executeTask(tasks):
    prompt = f"Tell me how to carry out these task: {tasks}, write the steps separately for each task"
    response = openai.Completion.create(engine="text-davinci-003",prompt=prompt,temperature=0.5,max_tokens=100,top_p=1,frequency_penalty=0,presence_penalty=0)
    result = response.choices[0].text.strip().split('\n')
    textResult = ""
    for i in result:
        textResult += i + '\n'
    return textResult

In [ ]:
# Use GPT3 to create a suitable query for Wolfram Alpha
def getWolfAlphaAnswers(questions):
    question_arr = questions.split("\n")
    result = ''
    for i in question_arr:
        if len(i) < 3:
            continue
        result += i + ' ' + wolfAlphaQuery(i) + '\n'
    return result

def wolframAlphaKnowledgeQuery(query):
    p1a = open('promptfa.txt','r')
    p1a = p1a.read()
    p1b_norm = open ('prompt1b_copy.txt','r')
    p1b_norm = p1b_norm.read()    
    firstprompt = str (p1a) + query + str(p1b_norm)
    response = openai.Completion.create(engine="text-davinci-003",prompt=firstprompt,temperature=0.5,max_tokens=100,top_p=1,frequency_penalty=0,presence_penalty=0)
    questions = response.choices[0].text.strip()
    out1 = getWolfAlphaAnswers(questions)
    return out1
    

In [ ]:
# Think about how to improve the correctness of our result

In [ ]:
# Answer checking
def gpt3_isCorrect(question, answer):
    Q = "Question: " + question + "\n"
    A = "Answer: " + answer + "\n\n"
    PROMPT = "Is the above answer appropriate for the above question, yes or no?"
    REGENERATE_PROMPT = " If not, suggest a change in the prompt to generate the appropriate answer."
    COMBINED = Q + A + PROMPT
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=COMBINED,
        temperature=0.0,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    txt_res = response.choices[0].text.strip()
    sliced = txt_res[0:3].lower()
    return sliced == 'yes'

# Test
# gpt3_isCorrect("1+2", "3")

In [ ]:
QUERY="Create a completed 3 day healthy meal plan for me"

FINAL = QUERY

In [ ]:
# Add wolfram alpha tool to do math and check how it runs
# Calculate first 100 prime numbers
# Find out how the logic is flowing

# Logic Flow:
# Seems to decide which tool to use based on name and description
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Web Search (SERP API)",
        func=search.run,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Wolfram Alpha",
        func=wolfAlphaQuery,
        description="useful for math calculations"
    ),
    Tool(
        name="Task Creation",
        func=createTaskList,
        description="useful for when you need to generate a list of tasks"
    ),
    Tool(
        name="Task Prioritization",
        func=prioritizeTaskList,
        description="useful for when you have a list of tasks that need to be prioritized"
    ),
    Tool(
        name="Task Execution",
        func=executeTask,
        description="useful for when you have a task that needs to be carried out"
    ),
    Tool(
        name="Wolfram Alpha Knowledge Query",
        func=wolframAlphaKnowledgeQuery,
        description="useful for when you want to obtain nutritional information"
    ),
]
agent = initialize_agent(tools, llm=gpt3, agent="zero-shot-react-description", verbose=True)
# Find out different agents

In [ ]:
result = agent.run(FINAL)
ITERS = 3
counter = 0

while counter < ITERS-1 and not gpt3_isCorrect(FINAL, result):
    print("Rerunning agent.run")
    result = agent.run(FINAL)
    counter += 1

print(result)